<a href="https://colab.research.google.com/github/xeniia-nikole/Python_detecting_objects/blob/master/searching_in_image_by_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install opencv-python
import cv2
import os
import numpy as np
from IPython.display import Image 

net = cv2.dnn.readNet('/content/yolov3-tiny.weights',
                '/content/yolov3-tiny.cfg')

with open('/content/coco.names') as f:
  labels = f.read().strip().split('\n')
#print(labels)

# image = cv2.imread('/content/couple_of_birds2.jpg')
# Image('/content/couple_of_birds2.jpg')

image = cv2.imread('/content/couple2.jpg')
Image('/content/couple2.jpg')

height, width, depth = image.shape

In [ ]:
blob = cv2.dnn.blobFromImage(image, 1/255, (416, 416), (0, 0, 0), swapRB = True)
net.setInput(blob)

layers_names = net.getLayerNames()
unconnected_layers = net.getUnconnectedOutLayersNames()
print(unconnected_layers)

output_layers = net.forward(unconnected_layers)
print(output_layers)

In [128]:
def draw_objects(x, y, w, h, image):
  x1, y1 = x - w // 2, y - h // 2
  x2, y2 = x + w // 2, y + h // 2
  color = [238, 151, 255]
  width = 2
  font_size = 1
  font = cv2.FONT_HERSHEY_SIMPLEX
  text = 'bird'

  image = cv2.rectangle(image, (x1, y1), (x2, y2), color, width)
  image = cv2.putText(image, text, (x1, y1 - 10), font, font_size, color, width)

  return image 

In [135]:
boxes = []
probabilities = []

for layer in output_layers: 
  for result in layer:
    x, y, w, h = result[:4]
    x = int(x * width)
    y = int(y * height)
    w = int(w * width)
    h = int(h * height) 
    # print(layer)
    # print(result)
    
    probability = result[5:]
    index_max_probability = np.argmax(probability)
    max_probability = probability[index_max_probability]

    if max_probability > 0:
      image_with_objects = draw_objects(x, y, w, h, image)

      boxes.append([x, y, w, h])
      probabilities.append(max_probability)

In [ ]:
filtered_boxes = cv2.dnn.NMSBoxes(boxes, probabilities, 0.0, 0.7)
for x, y, w, h in filtered_boxes:
  image_with_objects = draw_objects(x, y, w, h, image)

In [ ]:
cv2.imwrite('/content/image_with_objects.jpg', image_with_objects)
Image('/content/image_with_objects.jpg')

In [132]:
os.remove('image_with_objects.jpg')
del(image_with_objects)